# Correlation (상관관계)
`06_corr.ipynb`

## 정의
- 두 변수간에 서로 얼마나 함께 변하는지 정도를 나타내는 통계 개념
- 값의 범위는 : -1 ~ +1
    - +1: 완벽한 양의 상관관계 -> x가 증가하면 y도 비례해서 증가 : 키 - 몸무게
    - 0: 상관 없음 : 키와 수학점수
    - -1: 완벽한 음의 상관관계 -> x가 증가하면 y가 비례해서 감소 : 운동량 - 체지방률

## 상관 계수 ($r$)
두 변수간의 관계가 얼마나 강한지 측정하는 법 (대표)
- 피어슨 (Pearson): 가장 많이 쓰임. 연속 변수 선형 관계 측정
- 스피어맨 (Spearman): 순위(랭크) 기반, 비 선형 관계, 이상치에 강함
- 켄달 (Kendall) : 순위 일관성 기반, 표본이 적을 때 안정적

In [ ]:
%pip install -q kagglehub[pandas-datasets]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


df = pd.read_csv('./OnlineRetail.csv', encoding='ISO-8859-1')

In [ ]:
from da_utils import profile, outliers, patterns

# 데이터 프로파일링
profile.get_data_profile(df)

In [ ]:
# 결측치 모니터링 함수 
patterns.analyze_missing_patterns(df)

In [ ]:
# 이상치 탐지 함수 (결측치 처리후 실행 필요)
outliers.outlier_detection(df, 0.999, iso_cont='auto', final_threshold=2)

In [ ]:
print('전체 거래', len(df))
print('고유 InvoiceNo', df['InvoiceNo'].nunique())
print('고유 고객수', df['CustomerID'].nunique())

전체 거래 541909
고유 InvoiceNo 25900
고유 고객수 4372


In [ ]:
# 데이터 전처리
df_clean = df.copy()

# CustomerID 결측값 제거 (고객 단위 분석을 위해 필수)
missing_customers = df_clean['CustomerID'].isnull().sum()
print(f'CustomerID 결측값 제거: {missing_customers}건')
df_clean = df_clean.dropna(subset=['CustomerID'])

# 취소 거래 분리 (InvoiceNo 가 'C' 로 시작)
cancel_mask = df_clean['InvoiceNo'].astype('str').str.startswith('C')
# 취소거래, 정상거래 분리
df_cancel = df_clean[cancel_mask]
df_clean = df_clean[~cancel_mask]
print(f'취소거래 분리: {len(df_cancel)}건')
print(f'정상 거래: {len(df_clean)}건')

# 파생 변수 생성 (상관 관계 분석)
# TotalAmount, InvoiceDate(DateTime), Year, Month, DayOfWeek(숫자), Hour, DayName(글자), CustomerID(Str)
df.head()

CustomerID 결측값 제거: 135080건
취소거래 분리: 8905건
정상 거래: 397924건


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
